In [160]:
import pickle

'''
#Biology dataset for question body + title
cs_ques_dict = pickle.load(open("../clustering/bio_ques_dict.dat","rb"))
cs_cluster_centers = pickle.load(open("../clustering/bio_cluster_centers.dat","rb"))
#cs_cluster_centers_indices = pickle.load(open("../clustering/bio_cluster_centers_indices.dat","rb"))
'''


#Biology dataset for question body + title +tag
cs_ques_dict = pickle.load(open("../clustering/bio_withtags_ques_dict.dat","rb"))
cs_cluster_centers = pickle.load(open("../clustering/bio_withtags_cluster_centers.dat","rb"))
#cs_cluster_centers_indices = pickle.load(open("../clustering/bio_cluster_centers_indices.dat","rb"))

'''
#Biology dataset for question title only
ques_dict = pickle.load(open("../clustering/bio_onlytitle_ques_dict.dat","rb"))
cluster_centers = pickle.load(open("../clustering/bio_onlytitle_cluster_centers.dat","rb"))
#cluster_centers_indices = pickle.load(open("../clustering/bio_onlytitle_cluster_centers_indices.dat","rb"))
'''

ans_dict = pickle.load(open("../clustering/biology_ans_dict.dat","rb"))
ans_cluster_centers = pickle.load(open("../clustering/biology_ans_cluster_centers.dat","rb"))
ans_cluster_centers_indices = pickle.load(open("../clustering/biology_ans_cluster_centers_indices.dat","rb"))

In [161]:
num_clusters = cluster_centers.shape[0]
print("Total number of clusters: ",num_clusters)

Total number of clusters:  1328


In [16]:
ans_num_clusters = ans_cluster_centers.shape[0]
print("Total number of answer clusters: ",ans_num_clusters)

Total number of answer clusters:  1402


In [162]:
#We have the cluster centers and also the label for each question
#For a given cluster, find its 10 nearby clusters, sorted according to their distances
#nearest first
#Use euclidean distance between the cluster centers for this
from sklearn.metrics.pairwise import euclidean_distances

def computeNearByClusters(cluster_centers):
    nearby_centers_dict = {}
    for i in range(len(cluster_centers)):
        #gives the distance of i-th center from all the centers
        distances = euclidean_distances(cluster_centers,[cluster_centers[i]])
        #sort these distances
        nearest_10_centers_array = distances.argsort(axis=0)[0:6]
        nearest_10_centers = []
        for elem in nearest_10_centers_array:
            nearest_10_centers.append(elem[0])
        if not i in nearby_centers_dict:
            nearby_centers_dict[i] = nearest_10_centers
    return nearby_centers_dict

In [163]:
#Dict where key = cluster_id, value = list of ids of 10 nearest clusters
nearby_centers_dict = computeNearByClusters(cluster_centers)

In [164]:
ans_nearby_centers_dict = computeNearByClusters(ans_cluster_centers)

In [165]:
print(nearby_centers_dict[0])

[0, 796, 698, 24, 1129, 460]


In [166]:
#Given a cluster, find all the questions/answers that belong to it
def findAllInCluster(num_clusters,dictionary):
    cluster_dict = {}
    for key, value in dictionary.items():
        if value[1] in cluster_dict:
            cluster_dict[value[1]].append(key)
        else:
            cluster_dict[value[1]] = [key]
    return cluster_dict
    

In [167]:
#dict where key = cluster_id
#value = list of all questions present in that cluster
questions_in_a_cluster_dict= {}
questions_in_a_cluster_dict = findAllInCluster(num_clusters,ques_dict)

In [168]:
answers_in_a_cluster_dict={}
answers_in_a_cluster_dict = findAllInCluster(ans_num_clusters,ans_dict)

In [169]:
print(questions_in_a_cluster_dict[0])
print(answers_in_a_cluster_dict[0])

[1, 1152, 7559, 8743, 9992, 20450, 21146, 36405, 43802, 46427, 48907, 56523, 67692, 69685, 70819, 71400, 72589, 73671, 80603, 80659]
[30, 59, 13544, 21280, 31736, 34234, 38526, 38785, 39405, 45158, 45636, 54510, 54525, 54978, 56742, 71871, 80622]


In [170]:
import math
#given a question embedding, calculate its magnitude
def computeMagnitude(embedding):
    square_sum = 0
    for i in range(len(embedding)):
        if not math.isnan(embedding[i]):
            square_sum = square_sum + (embedding[i]*embedding[i])
    return math.sqrt(square_sum)

def computeCosine(ques_1, ques_2):
    numerator = 0
    for i in range(len(ques_1)):
        if not math.isnan(ques_1[i]) and not math.isnan(ques_2[i]):
            numerator = numerator + ques_1[i]*ques_2[i]
    denominator = computeMagnitude(ques_1) * computeMagnitude(ques_2)
    return numerator/denominator

In [171]:
#Given a question, find which cluster it belongs to
#find all nearby clusters to that cluster
#retrieve all the questions from the nearby clusters, form a list of all the questions
#Do cosine similarity to find the questions most similar to the given question
#return top 10 of them

from sklearn.metrics.pairwise import cosine_similarity

def generateQuesRecommendations(ques_id):
    total_questions = []
    #input is question_id
    #cluster_label tells which cluster the question is in
    #ques_dict is a dictionary where key is ques_id and value is a tuple(embedding,label)
    cluster_label = ques_dict[ques_id][1]
    #find the 10 nearby clusters to a given cluster
    nearby_clusters = nearby_centers_dict[cluster_label]
    #nearby_clusters is a list
    for cluster in nearby_clusters:
        total_questions.extend(questions_in_a_cluster_dict[cluster])
    #total_questions is a list of all questions
    #for a given question find cosine similarity with all these questions
    cosine_scores = []
    for ques in total_questions:
        cosine_scores.append((ques,cosine_similarity(ques_dict[ques_id][0].reshape(1,-1),ques_dict[ques][0].reshape(1,-1))[0][0]))
    cosine_scores.sort(key=lambda x: x[1],reverse=True)
    return cosine_scores[0:100]

In [67]:
import pandas as pd

answersDF = pd.read_csv('../csv_files/biology_answers.csv')

answers ={}

#Building a dictionary of answerId with associated questionId
for index, row in answersDF.iterrows():
    answers[row['Id']] = row['ParentId']
    
#lookup parentID(Question ID) for answers
def lookupParentId(ansId):
    return answers[ansId]


In [68]:
def generateAnswerRecommendations(ans_id):
    total_answers = []
    #input is ans_id
    #cluster_label tells which cluster the question is in
    #ans_dict is a dictionary where key is ans_id and value is a tuple(embedding,label)
    cluster_label = ans_dict[ans_id][1]
    #find the 10 nearby clusters to a given cluster
    nearby_clusters = ans_nearby_centers_dict[cluster_label]
    #nearby_clusters is a list
    for cluster in nearby_clusters:
        total_answers.extend(answers_in_a_cluster_dict[cluster])
    #total_questions is a list of all questions
    #for a given question find cosine similarity with all these answers
    #finally lookup the question id of the similar answer and store it with the cosine similarity between the answers
    cosine_scores = []
    for ans in total_answers:
        que= lookupParentId(ans)
        cosine_scores.append((que,cosine_similarity(ans_dict[ans_id][0].reshape(1,-1),ans_dict[ans][0].reshape(1,-1))[0][0]))
    cosine_scores.sort(key=lambda x: x[1],reverse=True)
    return cosine_scores[0:100]

In [172]:
#For a given question, generate top 10 recommendations
#output is a dict, which has ques_id as key, and value as list of tuples
#where each tuple consists of ques_id and cosine similarity score
ques_recommendations = {}
for key, value in ques_dict.items():
    #here key represents the question_id
    #generate recommendation for each such question
    ques_recommendations[key] = generateQuesRecommendations(key)


In [80]:
ans_recommendations = {}
for key, value in ans_dict.items():
    #here key represents the question_id based on answer embedding
    #generate recommendation for each such question
    #Based on answer similarities, find question recommendations
    que= lookupParentId(key)
    if que in ans_recommendations.keys():
        ans_recommendations[que].extend(generateAnswerRecommendations(key))
    else:
        ans_recommendations[que] = generateAnswerRecommendations(key)

In [173]:
def combine(ans_recommendations,ques_recommendations):
    ques_ans_recommendations = {}
    for key,val in ans_recommendations.items():
        if key in ques_ans_recommendations.keys():
            ques_ans_recommendations[key].extend(ans_recommendations[key])
        else:
            ques_ans_recommendations[key] = ans_recommendations[key]
    for key,val in ques_recommendations.items():
        if key in ques_ans_recommendations.keys():
            ques_ans_recommendations[key].extend(ques_recommendations[key])
        else:
            ques_ans_recommendations[key] = ques_recommendations[key]
    return ques_ans_recommendations
            
            

In [174]:
combinedRecommendations = combine(ans_recommendations,ques_recommendations)

In [175]:
def Sort_Tuple(tup):   
    return(sorted(tup, key = lambda x: x[1],reverse=True))

for key,value in combinedRecommendations.items():
    temp = []
    for tup in combinedRecommendations[key]:
        if(key==tup[0]):
            continue
        else:
            temp.append(tup)
    combinedRecommendations[key]=set(temp)
    t = Sort_Tuple(combinedRecommendations[key])
    combinedRecommendations[key] = t

In [141]:
combinedRecommendations[90]

[(19316, 0.9747380994367993),
 (26214, 0.9727712132934951),
 (58156, 0.9720743798861091),
 (29663, 0.9675041486829914),
 (24786, 0.9670751735112086),
 (39134, 0.9669219605622068),
 (19316, 0.9665601254085994),
 (26112, 0.9654741542054411),
 (24786, 0.964785864013664),
 (44218, 0.9647804968993601),
 (57521, 0.9646207525428151),
 (40714, 0.9640190765374457),
 (43111, 0.9634420772956083),
 (3246, 0.9629902021869791),
 (49201, 0.9622910684467376),
 (78669, 0.9614967341571361),
 (80353, 0.9605827250859331),
 (73586, 0.9601614596510477),
 (287, 0.9591773370114542),
 (73844, 0.958988172630775),
 (72832, 0.9589729538214032),
 (78450, 0.9588816119379051),
 (287, 0.9587748586167355),
 (25903, 0.958500768690979),
 (79633, 0.9581984603118905),
 (20537, 0.9576059677622438),
 (80353, 0.956804418282188),
 (58867, 0.9558923660647323),
 (64616, 0.9558554991878175),
 (46243, 0.9556018571160875),
 (57284, 0.9555105750407827),
 (36229, 0.9553985392057636),
 (20674, 0.9544625407849148),
 (19590, 0.95419812

In [140]:
#Biology dataset - Question body and title
pickle.dump(ques_recommendations,open("ques_recommendations.dat","wb"))
pickle.dump(ans_recommendations,open("ans_recommendations.dat","wb"))
pickle.dump(combinedRecommendations,open("que_ans_recommendations.dat","wb"))

In [158]:
#Biology dataset - only title
pickle.dump(ques_recommendations,open("title_ques_recommendations.dat","wb"))
pickle.dump(combinedRecommendations,open("title_que_ans_recommendations.dat","wb"))

In [177]:
#Biology dataset - tag+title+body
pickle.dump(ques_recommendations,open("w_tags_ques_recommendations.dat","wb"))
pickle.dump(combinedRecommendations,open("w_tags_que_ans_recommendations.dat","wb"))